## Settaggio Ambiente

Facciamo le istallazioni delle librerie utili

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install re
!pip install nltk
!pip install numpy
!pip install pickle

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Importiamo ora le librerie necessarie alla nostra analisi

In [3]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import PorterStemmer

nltk.download('punkt')
import seaborn as sns
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Esplorazione dataset

Carichiamo i dati

In [4]:
reviews = pd.read_csv("/content/drive/MyDrive/Assigment4/Reviews.csv")

Andiamo a svolgere alcune analisi esplorative

In [5]:
reviews.shape

(568454, 10)

Il dataset è formato da 568454 recensioni e 10 attributi

In [6]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Sembra essere stato letto tutto in maniera corretta :)

In [7]:
reviews.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

Vediamo che ci sono 40 reviews che presentano alcuni valori nulli, dato che sono pochi procediamo a rimuoverli dal dataset

In [8]:
reviews = reviews.dropna()
print(reviews.shape)

(568411, 10)


In [9]:
subset = {"UserId", "ProfileName", "Time", "Text"}
rev = reviews.drop_duplicates(subset=subset, keep="first")
reviews.shape

(568411, 10)

Procediamo a rimuovere le colonne che non saranno utili per il task di summarization

In [10]:
reviews = reviews.drop(['Id','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)


In [11]:
reviews.head()

,ProductId,Summary,Text
0,B001E4KFG0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,Great taffy,Great taffy at a great price. There was a wid...


Andiamo a visualizzare meglio il testo contenuto all' interno della variabile "summary" e nella variabile "text"

In [ ]:
for i in range(6):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print(reviews.Text[i])
    print()

Review # 1
Good Quality Dog Food
I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

Review # 2
Not as Advertised
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

Review # 3
"Delight" says it all
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces 

In [ ]:
#Analysis of number of sentences
text_lens_ = [len(sent_tokenize(str(reviews['Text'][i]))) for i in range(len(reviews)-1)]
summary_lens_ = [(len(sent_tokenize(str(reviews['Summary'][i])))) for i in range(len(reviews)-1)]
graph_df_ = pd.DataFrame()
graph_df_['text'] = text_lens_
graph_df_['summary'] = summary_lens_
#Plot distribution of number of sentences per review
figdist = sns.distplot(graph_df_.text,axlabel="Number of sentences per review")
fig = figdist.get_figure()

KeyboardInterrupt: ignored

In [ ]:
#Analysis of number of words per text
text_lens = [len(word_tokenize(str(reviews['Text'][i]))) for i in range(len(reviews)-1)]
summary_lens = [(len(word_tokenize(str(reviews['Summary'][i])))) for i in range(len(reviews)-1)]
graph_df = pd.DataFrame()
graph_df['text'] = text_lens
graph_df['summary'] = summary_lens
#Plot distribution of number of words per review
figdist = sns.distplot(graph_df.text).set_title('Number of words per review')
fig = figdist.get_figure()

Andiamo a fare un' analisi grafica del numero di parole e del numero di frasi per ogni testo delle review

# Text Preprocessing 

Andiamo ora a testare diversi approcci di pre-processing per vedere poi quello che ci darà dei risultati migliori!


## Text Preprocessing - 0

In [12]:
#List of contractions to transform
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}


#Function to transform the contraction in the extended form
def rem_contractions(x):
  x = str(x).split()
  new_text = []
  for word in x:
    if word in contractions:
      new_text.append(contractions[word])
    else:
      new_text.append(word)
  return ' '.join([str(elem) for elem in new_text])

#Function to apply preprocessing operations
def text_preprocessing(x,remove_stopwords=True):
  cleaned = rem_contractions(x)
  cleaned = re.sub(r"'s\b","", cleaned) #remove genitives
  cleaned = pd.Series(cleaned).str.replace("[^a-zA-Z]", " ") # remove punctuation, special characters and numbers
  cleaned = [s.lower() for s in cleaned] #to lowercase
  if remove_stopwords: 
    stops = set(stopwords.words("english"))
    cleaned = " ".join([i for i in cleaned if i not in stops]) 
  #No stemming because it could cause a loss of information
  return cleaned

## Text Preprocessing - 1

Questo primo pre-processing non andrà a tenere in conto la variabile productID

In [13]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions: #su un dizionario si itera sulla key
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text) #ripuliamo il text ponendolo uguale a " " e li andiamo ad aggiungere il nuovo testo 
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Possiamo rimuovere gli stopwords
   
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

## Seq2Seq

In [34]:
from nltk.compat import remove_accents
import re
from nltk.corpus import stopwords
stop = stopwords.words('english') 

# Remove non-alphabetic characters (Data Cleaning)
def text_strip(column,remove_stopword=True):

    for row in column:
        row = re.sub("(\\t)", " ", str(row)).lower()
        row = re.sub("(\\r)", " ", str(row)).lower()
        row = re.sub("(\\n)", " ", str(row)).lower()

        # Remove _ if it occurs more than one time consecutively
        row = re.sub("(__+)", " ", str(row)).lower()

        # Remove - if it occurs more than one time consecutively
        row = re.sub("(--+)", " ", str(row)).lower()

        # Remove ~ if it occurs more than one time consecutively
        row = re.sub("(~~+)", " ", str(row)).lower()

        # Remove + if it occurs more than one time consecutively
        row = re.sub("(\+\++)", " ", str(row)).lower()

        # Remove . if it occurs more than one time consecutively
        row = re.sub("(\.\.+)", " ", str(row)).lower()

        # Remove the characters - <>()|&©ø"',;?~*!
        row = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", " ", str(row)).lower()

        # Remove mailto:
        row = re.sub("(mailto:)", " ", str(row)).lower()

        # Remove \x9* in text
        row = re.sub(r"(\\x9\d)", " ", str(row)).lower()

        # Replace INC nums to INC_NUM
        row = re.sub("([iI][nN][cC]\d+)", "INC_NUM", str(row)).lower()

        # Replace CM# and CHG# to CM_NUM
        row = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", "CM_NUM", str(row)).lower()

        # Remove punctuations at the end of a word
        row = re.sub("(\.\s+)", " ", str(row)).lower()
        row = re.sub("(\-\s+)", " ", str(row)).lower()
        row = re.sub("(\:\s+)", " ", str(row)).lower()

        # Replace any url to only the domain name
        try:
            url = re.search(r"((https*:\/*)([^\/\s]+))(.[^\s]+)", str(row))
            repl_url = url.group(3)
            row = re.sub(r"((https*:\/*)([^\/\s]+))(.[^\s]+)", repl_url, str(row))
        except:
            pass

        # Remove multiple spaces
        row = re.sub("(\s+)", " ", str(row)).lower()

        # Remove the single character hanging between any two spaces
        row = re.sub("(\s+.\s+)", " ", str(row)).lower()
        if remove_stopword:
          column = column.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

        

        yield row

In [35]:
processed_text = text_strip(reviews['Text'])
processed_summary = text_strip(reviews['Summary'])


In [ ]:
import spacy
from time import time

nlp = spacy.load('en', disable=['ner', 'parser']) 

# Process text as batches and yield Doc objects in order
text = [str(doc) for doc in nlp.pipe(processed_text, batch_size=5000)]

summary = ['_START_ '+ str(doc) + ' _END_' for doc in nlp.pipe(processed_summary, batch_size=5000)]

In [17]:
reviews.head()

,ProductId,Summary,Text
0,B001E4KFG0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,Great taffy,Great taffy at a great price. There was a wid...


In [18]:
processed_text

'  i have bought several of the vitality canned d      product arrived labeled as jumbo salted peanut      this is a confection that has been around a fe      if you are looking for the secret ingredient i      great taffy at a great price  there was a wid               great for sesame chicken  this is a good if no           i am disappointed with the flavor  the chocolat           these stars are small  so you can give       o           these are the best treats for training and rew           i am very satisfied  product is as advertised     name  text  length          dtype  object'

## Latent Semantic Analysis (LSA)



In [ ]:
def weighting(input,weight):
  #Preprocess the input
  sentences_cleaned = [clean_text(y) for y in (sent_tokenize(input))]
  DFs = []
  if weight == "f":
    for sent in sentences_cleaned:
      tokens = nltk.word_tokenize(sent)
      tokens2 = pd.DataFrame(tokens)
      tokens2.columns=['Frequency']
      tokens2 = pd.DataFrame(tokens2.Frequency.value_counts())
      tokens2 = tokens2.reset_index()
      tokens2.columns = ['Word','Frequency']
      DFs.append(tokens2)
      df = DFs[0]
      i=0
      for elem in DFs:
        df = pd.merge(df,elem,on="Word",how="outer")
      df.columns = [str(i) for i in range(len(df.columns))]
      df = df.drop('1',axis=1)
      df = df.replace(np.nan,0)
      df.set_index("0",inplace=True)
      df.columns = [str(i) for i in range(len(df.columns))]
      df = df.values
    return df
  elif weight == "tf":
    for sent in sentences_cleaned:
      tokens = nltk.word_tokenize(sent)
      length = len(tokens)
      tokens2 = pd.DataFrame(tokens)
      tokens2.columns=['TF']
      tokens2 = pd.DataFrame(tokens2.TF.value_counts()/length)
      tokens2 = tokens2.reset_index()
      tokens2.columns = ['Word','TF']
      DFs.append(tokens2)
      df = DFs[0]
      i=0
      for elem in DFs:
        df = pd.merge(df,elem,on="Word",how="outer")
      df.columns = [str(i) for i in range(len(df.columns))]
      df = df.drop('1',axis=1)
      df = df.replace(np.nan,0)
      df.set_index("0",inplace=True)
      df.columns = [str(i) for i in range(len(df.columns))]
      df = df.values
    return df
  elif weight == 'tf.idf':
    for sent in sentences_cleaned:
      tokens = nltk.word_tokenize(sent)
      length = len(tokens)
      tokens2 = pd.DataFrame(tokens)
      tokens2.columns=['TFIDF']
      tokens2 = pd.DataFrame(tokens2.TFIDF.value_counts()/length)
      tokens2 = tokens2.reset_index()
      tokens2.columns = ['Word','TFIDF']
      DFs.append(tokens2)
      df = DFs[0]
      i=0
      for elem in DFs:
        df = pd.merge(df,elem,on="Word",how="outer")
      df.columns = [str(i) for i in range(len(df.columns))]
      df = df.drop('1',axis=1)
      df = df.replace(np.nan,0)
      df.set_index("0",inplace=True)
      df.columns = [str(i) for i in range(len(df.columns))]
      num = len(df.columns)
      rows = []
      for index in df.index:
        den = np.count_nonzero(df.loc[index])
        idf = num/den
        idf = np.log(idf)
        row = df.loc[index]*idf
        rows.append(row)
    return pd.DataFrame(rows).values

def SVD_weight(input,weight):
  df = weighting(input,weight)
  U, S, VT = np.linalg.svd(df, full_matrices=0)
  return pd.DataFrame(VT)

def ranking_sent(input,weight):
  matrix = SVD_weight(input,weight)
  score = {}
  for i in matrix.columns:
    score[i]=np.sqrt(np.sum(matrix[i]**2))
  return score

def LSA_summarize(input,num_of_sentences,weight="tf.idf"):
  ranking = ranking_sent(input,weight)
  sentences = sent_tokenize(input)
  if num_of_sentences > len(sentences):
    print("ERROR: There are only",len(sentences),"sentences in the text.")
  else:
    s = pd.DataFrame(pd.concat((pd.Series(sentences),pd.Series(ranking)),axis=1))
    s.columns = ['Sentence','Score']
    s = s.sort_values("Score",ascending = False)
    summary_sentences = [i for i in s['Sentence'][:num_of_sentences]]
    return ' '.join([str(elem) for elem in summary_sentences])

In [ ]:
#Example with one review
for i in range(3):
  summ = LSA_summarize(reviews['Text'][i],1)
#Example with one review
  print("--------------------------------------")
  print()
  print("Review number", i)
  print()
  print("Review:","\n",reviews['Text'][i])
  print()
  print("Original summary:","\n",reviews['Summary'][i])
  print()
  print("Predicted summary:","\n",summ)
  print()



--------------------------------------

Review number 0

Review: 
 I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

Original summary: 
 Good Quality Dog Food

Predicted summary: 
 I have bought several of the Vitality canned dog food products and have found them all to be of good quality.

--------------------------------------

Review number 1

Review: 
 Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

Original summary: 
 Not as Advertised

Predicted summary: 
 Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted.

--------------------------------------

Review number 2

Review: 
 This is 

## TextRank

In [ ]:
#Load GloVe for text representation
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-01-04 16:29:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-01-04 16:29:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-04 16:29:17--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
#Function to get the summary
def summary(x,N,pretty=False):
  #Split into preprocessed and not preprocessed sentences
   sentences_cleaned = [text_preprocessing(y) for y in (sent_tokenize(x))]
   sentences = [y for y in (sent_tokenize(x))]
  #Text representation
   sentence_vectors = [] 
   for i in sentences_cleaned:
     if len(i) != 0:
       v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
     else:
       v = np.zeros((100,))
     sentence_vectors.append(v)
   #Compute similarity matrix
   sim_mat = np.zeros([len(sentences), len(sentences)])
   for i in range(len(sentences)):
     for j in range(len(sentences)):
       if i != j:
         sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
   #Convert similarity matrix into a graph
   nx_graph = nx.from_numpy_array(sim_mat)
   scores = nx.pagerank(nx_graph)
   #Rank sentences
   ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
   #Select top N sentences
   summary=[]
   for i in range(N):
     summary.append(ranked_sentences[i][1])
   if pretty:
     return ' '.join([str(elem) for elem in summary]) #Return the readable text
   else:
     return summary #Return the list of summary sentences

In [ ]:
#Example with one review
#Example with one review
print("Review:","\n",reviews['Text'][2])
print()
print("Original summary:","\n",reviews['Summary'][2])
print()
print("Predicted summary:","\n",summary(reviews['Text'][2],1,pretty=True))

Review: 
 This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.

Original summary: 
 "Delight" says it all

Predicted summary: 
 And it is a tiny mouthful of heaven.


In [ ]:
#Install requirements and BERT summarizer
!pip install spacy
!pip install transformers  
!pip install bert-extractive-summarizer
#Import BERT Summarizer 
from summarizer import Summarizer

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 453 kB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

#Import BERT Summarizer 
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
def Bert_summarizer(text):
  bert_model = Summarizer()
  outp= bert_model(text)
  bert_summary = ''.join(outp)
  s = sent_tokenize(bert_summary)
  return s[0]

In [ ]:
print("Review:","\n",reviews['Text'][1003])
print()
print("Original summary:","\n",reviews['Summary'][1003])
print()
print("Predicted summary:","\n",Bert_summarizer(reviews['Text'][1003]))

Review: 
 These little animal crackers taste good & my doggies like them, but I won't be buying again. They are like the human animal crackers & have flour & sugar & other bad stuff for dogs. I will continue to buy the healthy treats for my doggies which they also love without sugar. You might want to try Wellness, Blue Buffalo, Natural Balance, Zukes, just to name a few.

Original summary: 
 Not healthy but they taste good



Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Predicted summary: 
 These little animal crackers taste good & my doggies like them, but I won't be buying again.


## GPT2

In [ ]:

GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
def GPT2_summarizer(text):
  outp= GPT2_model(text)
  GPT2_summary = ''.join(outp)
  s = sent_tokenize(GPT2_summary)
  return s[0]

In [ ]:
print("Review:","\n",reviews['Text'][1003])
print()
print("Original summary:","\n",reviews['Summary'][1003])
print()
print("Predicted summary:","\n",GPT2_summarizer(reviews['Text'][1003]))

Review: 
 These little animal crackers taste good & my doggies like them, but I won't be buying again. They are like the human animal crackers & have flour & sugar & other bad stuff for dogs. I will continue to buy the healthy treats for my doggies which they also love without sugar. You might want to try Wellness, Blue Buffalo, Natural Balance, Zukes, just to name a few.

Original summary: 
 Not healthy but they taste good

Predicted summary: 
 These little animal crackers taste good & my doggies like them, but I won't be buying again.


Andiamo a risolvere quelle che sono le abbreviazioni in inglese (aren't ---> are not) tramite il seguente dizionario trovato al seguente link:
 http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

Decidiamo di proseguire rimuovendo le stop words per i testi così da renderli più semplicemente allenabili da un modello, mentre per le summary le manteniamo così da renderle più simili ad una frase naturale e per poi andarle a confrontare meglio con quelle da noi prodotte.

In [ ]:

clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))


Andiamo ad ispezionare il testo ripulito e confrontiamolo con quello vecchio!

In [ ]:
for i in range(3):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print(reviews.Text[i])
    print()
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

    

Review # 1
Good Quality Dog Food
I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

Clean Review # 1
good quality dog food
bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better

Review # 2
Not as Advertised
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

Clean Review # 2
not as advertised
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo

Review # 3
"Delight" says it all
This is a confection that has been around a few centuries.  It is a light

Definiamo una funzione che va a contare il numero di volte in cui una parola compare in un determinato testo

In [ ]:
def count_words(count_dict, text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1 
            else:
                count_dict[word] += 1

Andiamo ora sia per le summary sia per i text ad andare ad indagare quanto ogni parola si è presentata e definiamo anche la grandezza del nostro dizionario

In [ ]:
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Nel vocabolario ci sono:", len(word_counts),"parole")

Nel vocabolario ci sono: 132884 parole


Procediamo con il word embedding, per svolgere questo task abbaimo usato numbertach (https://github.com/commonsense/conceptnet-numberbatch)

In [ ]:
embeddings_index = {}
with open('s171WJJHrL4M', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Nel Word embeddings ci sono:', len(embeddings_index),"parole")

FileNotFoundError: ignored

Vediamo che ogni parola viene annidata utilizzando 300 features (qui abbiamo preso per esempio la parola heaven)

In [ ]:
len(embeddings_index[ "heaven"])

Cerchiamo il numero di parole del nostro vocabolario che non sono presenti attraverso il nostro word embedding, poniamo però una soglia a 20 perchè devono essere abbastanza presenti nelle recensioni in modo che il modello possa capire il loro significato

In [ ]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Limitiamo le parole che utilizzeremo nel nostro vocabolario a quelle che hanno una soglia maggiore di 20 o a quelle che troviamo in GloVe

In [ ]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

#Aggiungiamo questi token speciali al nostro vocabolario, saranno utili quando useremo seq2seq
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

In [ ]:
#Utilizziamo la dimensione di 300 perchè era quella utilizzata nel nostro precedente transformer
embedding_dim = 300
nb_words = len(vocab_to_int)

# Creiamo una matrice di zeri di dimensione paroledelvocabolario x la dimensione dell embedding
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # Se le parole non erano presenti nel nostro precedente transformer procediamo a fare un random embedding
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

Vediamo che tutte le parole che volevamo usare sono state usate

Definiamo la funzione che riformatta il nostro testo mantenendo solo le parole che abbiamo mantenuto nel nostro nuovo vocabolario e che aggiunge i tokens come sotto riportato

Se la parola non è presente nel nostro vocabolario la sostituiamo con il token <"UNK"> che ha proprio il compito di essere utilizzato per sostituire le parole rare che non sono presenti nel nostro vocabolario.

Aggiungiamo il toke <"EOS"> alla fine di ogni frase.

In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

Andiamo ora ad eseguirla (per il momento lasciamo settato eos=False, andremmo poi in seguito a verificare se vi sarà un cambiamento) sulle nostre summaries e texts ripuliti

In [ ]:
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words:", word_count)
print("Total number of UNKs:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Quello che vogliamo fare ora è rimuovere le recensioni che contengono troppi UNK (valori rari e non riconosciuti) in quanto potrebbero essere recensioni scritte in altre lingue, altra cosa che vogliamo fare è rendere le recensioni in un range di lunghezza comune

In [ ]:
#definiamo una funzione che crea un dataframe che contiene per ogni testo la sua lunghezza
def create_lengths(text):
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
#andiamo a calcolare la lunghezza sia per le summaries che dei text e ne plottiamo alcune statistiche descrittive
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

In [ ]:
#vediamo che i max sono molto distanti dal terzo quartile, andiamo quindi ad indagare altri percentili
print("90-esimo,95-esimo,99-esimo percentili per i Texts: ")
print()
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))
print()
print("90-esimo,95-esimo,99-esimo percentili per i Summaries: ")
print()
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))



In [ ]:
#definiamo una funzione per contare il numero di unk presenti in ogni frase
def unk_counter(sentence):
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

Andiamo a ordinare le summaries e i relativi testi per la lunghezza del testo (dal più piccolo al più grande)
Limitiamo la lunghezza che possono avere le summaries e i testi al 99-esimo percentile
E rimuoviamo le recensioni che contengono troppi unks sia nei testi che nelle summaries

In [ ]:
sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2 #per le summaries lo fissiamo almeno ad una lunghezza di 2
unk_text_limit = 1
unk_summary_limit = 0
print(min(lengths_texts.counts))
for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# compariamo le lunghezze per essere sicuri che coincidano
print(len(sorted_summaries))
print(len(sorted_texts))

In [ ]:
import pickle
with open("/content/drive/MyDrive/Assigment4/sorted_summaries.txt", "rb") as fp:   #Pickling
  sorted_summaries=pickle.load(fp)

import pickle
with open("/content/drive/MyDrive/Assigment4/sorted_texts.txt", "rb") as fp:   #Pickling
  sorted_texts=pickle.load(fp)

EOFError: ignored